In [1]:
!apt-get update
!apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip install JPype1
!pip install konlpy
!JAVA_HOME="C:\Program Files\Java\jdk-19"


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [913 kB]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:13 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,990

In [2]:
import konlpy

from konlpy.tag import Okt

from konlpy.utils import pprint

okt = Okt()

# 새 섹션

In [3]:
!pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 1.8 MB/s 
  Using cached pybind11-2.10.0-py3-none-any.whl (213 kB)
  Created wheel for fasttext: filename=fasttext-0.9.2-cp37-cp37m-linux_x86_64.whl size=3162184 sha256=1ca70156bd692eae9c298636b9b4944b87173a4fcdbfc7c7569bbf63be4606f0
  Stored in directory: /root/.cache/pip/wheels/4e/ca/bf/b020d2be95f7641801a6597a29c8f4f19e38f9c02a345bab9b
Successfully built fasttext


In [4]:

import numpy as np
import tensorflow as tf

np.random.seed(42)
tf.random.set_seed(42)

In [5]:
from konlpy.tag import *
import fasttext
import pandas as pd
import re
from tqdm import tqdm_notebook
import numpy as np
from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input, Bidirectional
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras import layers
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
from konlpy.tag import Okt
okt = Okt()


데이터셋 불러오기

자료는 공공 데이터 포탈에서 참고했습니다.

In [7]:
import pandas as pd

In [8]:
df_corona = pd.read_csv('/content/drive/MyDrive/Section4 Project/corona.csv')
df_culture = pd.read_csv('/content/drive/MyDrive/Section4 Project/culture.csv', encoding='cp949')
df_fake = pd.read_csv('/content/drive/MyDrive/Section4 Project/fake_news.csv')

In [8]:
df_corona.head()

,주소,일자,언론사,기고자,제목,통합 분류1,통합 분류2,통합 분류3,사건_사고 분류1,사건_사고 분류2,사건_사고 분류3,개체명(인물),개체명(지역),개체명(기업기관),키워드,특성추출,본문,원본주소
0,http://www.bigkinds.or.kr/news/newsDetailView....,2021-07-01,디지털타임스,박재찬,폭염에 올 여름 전력수요 사상 최고치 전망...삐끗하면 `정전` 우려,경제>자원,경제>유통,경제>자동차,NaN,NaN,NaN,NaN,"경기, 고성하이화력, 부산, 태양광","국무총리, 산업부, 산업통상자원부, 전력거래소, 정부","폭염,전망,여름,전력,수요,사상,최고,정전,올여름,폭염,영향,산업,생산,증가,전기수...","여름철,올여름,산업부,전력수요,gw,예비력,대책기간,상한전망,발전기,전력판매량,최고...",올여름 폭염과 산업생산 증가 영향으로 전기수요가 역대 최고치를 기록할 것이라는 전망...,http://www.dt.co.kr/contents.html?article_no=2...
1,http://www.bigkinds.or.kr/news/newsDetailView....,2021-07-01,디지털타임스,은진,상반기 수출 3000억달러 돌파…반도체·친환경 날개 달고 올해 `역대급` 실적 전망,경제>무역,경제>유통,경제>증권_증시,NaN,NaN,NaN,문승욱,"미국, 아세안, 인도, 일본, 중국, 중남미, 중동","CIS, EU, KOTRA, 무역협회, 산업부, 산업연구원, 산업통상자원부, 석유화...","3000억,상반기,수출,달러,돌파,반도체,친환경,날개,역대급,실적,전망,수출,상반기...","반도체,상반기,수출액,친환경,3000억,원자재,기저효과,우리나라,경신",올해 상반기 우리나라 수출이 누적 기준 사상 처음으로 3000억달러를 돌파했다......,http://www.dt.co.kr/contents.html?article_no=2...
2,http://www.bigkinds.or.kr/news/newsDetailView....,2021-07-01,디지털타임스,강민성,"나라빚 1000조인데, 채무상환 고작 2조…전문가들 재정정상화 생색내기",경제>경제일반,NaN,NaN,NaN,NaN,NaN,"박영범, 신세돈, 홍남기",NaN,"경제학과, 국가재정법, 기획재정부, 숙명여대, 정부, 한성대","나라빚,채무상환,2조,전문가들,재정정상화,정부,초과,세수,예상,세계,잉여금,기금재원...","국가채무,2조,1000조,전문가,33조,채무상환,전문가들,본예산,한성대,홍남기,숙명...","정부가 올해 초과 세수 예상분 31조5000억원에 세계 잉여금, 기금재원까지 끌어모...",http://www.dt.co.kr/contents.html?article_no=2...
3,http://www.bigkinds.or.kr/news/newsDetailView....,2021-07-01,디지털타임스,강민성,국세청 신임 차장에 임광현,지역>경남,지역>대구,지역>대전,NaN,NaN,NaN,"김재철, 김창기, 임광현, 임성빈","경북, 봉화, 장흥, 전남, 충남, 홍성","국세청, 부산지방국세청, 부산청장, 서울지방국세청, 서울지방국세청장, 서울청, 중부...","차장,국세청,신임,임광현,임광현,서울지방,국세청장,차장,국세청,임명,서울,청장,자리...","국세청,서울지방국세청,임성빈,임광현,국세청장,부산,납세자,서울지방,서울지방국세청장,...",임광현 서울지방국세청장이 차기 국세청 차장으로 임명됐다... 서울청장 자리에는 임성...,http://www.dt.co.kr/contents.html?article_no=2...
4,http://www.bigkinds.or.kr/news/newsDetailView....,2021-07-01,디지털타임스,강민성,국세청 차장에 임광현…서울청장에 임성빈,지역>경남,지역>대구,지역>대전,NaN,NaN,NaN,"김재철, 김창기, 임광현, 임광현…서울청장, 임성빈","경북, 봉화, 장흥, 전남, 충남, 홍성","국세청, 부산지방국세청, 부산청장, 서울지방국세청, 서울지방국세청장, 서울청, 중부...","국세청,차장,임광현,서울청장,임성빈,국세청,인사,고위,정기,임광현,서울지방,국세청장...","국세청,임성빈,서울지방국세청,임광현,국세청장,부산,납세자,서울청장,서울지방,서울지방...",임광현 서울지방국세청장이 차기 국세청 차장으로 임명됐다... 서울청장 자리에는 임성...,http://www.dt.co.kr/contents.html?article_no=2...


In [ ]:
df_corona['label']= 0
df_culture['label']=0
df_fake['label']=1

df_1 = pd.concat([df_corona, df_culture])
df = pd.concat([df_1,df_fake])
df

In [10]:
df_test = df[['제목','본문', 'label']]

필요한 데이터만 뽑아내기 및 불용어 처리

In [11]:
def get_stopwords():
    stopwords = list()
    
    f = open('/content/drive/MyDrive/Section4 Project/stopwords.txt', 'r', encoding='utf-8')
    
    while True:
        line = f.readline()
        if not line: break
        stopwords.append(line.strip())
        
    return stopwords

def okt_tokenizer(text):
    okt = Okt()
    text = re.sub(r'[^ ㄱ-ㅣ가-힣A-Za-z]', '', text) # 특수기호 제거
    stopwords = get_stopwords() # 불용어
    
    return [token for token in okt.nouns(text)
            if len(token) > 1 and token not in stopwords]

데이터 TF_IDF 사용하여 유사도 확인

In [12]:
import pathlib
import random
import string
import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
import spacy
from spacy.tokenizer import Tokenizer
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors

In [14]:
tfidf = TfidfVectorizer(max_features=300, tokenizer= okt_tokenizer)

결측치 제거

In [17]:
df_test = df_test.dropna()

In [ ]:
tdm = tfidf.fit_transform(df_test['본문'])

In [ ]:
word_count = pd.DataFrame({
    '단어':  CountVectorizer.get_feature_names(),
    'tf-idf': tdm.sum(axis=0).flat
})

In [ ]:
word_count.sort_values('tf-idf', ascending=False).head(5)

In [13]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import precision_score, recall_score
X_train, X_test, y_train, y_test = train_test_split(
    df_test, df_test['label'], test_size=0.2, random_state= 42)

In [ ]:
embedding_dim = 128
hidden_units = 128

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, mask_zero=True))
model.add(Bidirectional(LSTM(hidden_units, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation=('softmax'))))

model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])
model.fit(X_train, y_train, batch_size=128, epochs=7, validation_data=(X_test, y_test))